## Convolutional Neural Network Using Tensorflow on Custom Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path =os.listdir('dataset')
room_types=os.listdir('dataset')
print(room_types)
print("Types of rooms found: ",len(dataset_path))


['bed_room', 'dining_room', 'living_room']
Types of rooms found:  3


In [2]:
rooms = []

for item in room_types:
    all_rooms = os.listdir('dataset' + '/' + item)
    for room in all_rooms:
        rooms.append((item, 'dataset' + '/' + item + '/' + room))
        print(rooms[:1])


[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340.jpg')]
[('bed_room', 'dataset/bed_room/bed-1303451__340

In [3]:
rooms_df=pd.DataFrame(data=rooms,columns=['room type', 'image'])
print(rooms_df.head())
print(rooms_df.tail())

  room type                                  image
0  bed_room  dataset/bed_room/bed-1303451__340.jpg
1  bed_room  dataset/bed_room/bed-1846251__340.jpg
2  bed_room  dataset/bed_room/bed-3786264__340.jpg
3  bed_room  dataset/bed_room/bed-4065946__340.jpg
4  bed_room  dataset/bed_room/bed-4343379__340.jpg
       room type                                              image
98   living_room  dataset/living_room/pexels-rachel-claire-54903...
99   living_room  dataset/living_room/pexels-rachel-claire-54903...
100  living_room  dataset/living_room/pexels-rachel-claire-81129...
101  living_room  dataset/living_room/pexels-taryn-elliott-95657...
102  living_room  dataset/living_room/pexels-taryn-elliott-95659...


In [4]:
print("Total number of rooms in the dataset:" ,len(rooms_df))

Total number of rooms in the dataset: 103


In [5]:
room_count = rooms_df['room type'].value_counts()
print("rooms in each category: ")
print(room_count)

rooms in each category: 
room type
living_room    39
bed_room       37
dining_room    27
Name: count, dtype: int64


In [6]:
import cv2
path = 'dataset/'

im_size=60

images=[]
labels=[]

for i in room_types:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path)]

    for f in filenames:
        img=cv2.imread(data_path +'/'+f)
        img=cv2.resize(img,(im_size,im_size))
        images.append(img)
        labels.append(i)

In [7]:
images = np.array(images)

images.shape

(103, 60, 60, 3)

In [8]:
images = images.astype('float32')/255.0

In [9]:
images.shape

(103, 60, 60, 3)

In [10]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [11]:
y=rooms_df['room type'].values
print(y[:5])

['bed_room' 'bed_room' 'bed_room' 'bed_room' 'bed_room']


In [12]:
y_labelencoder = LabelEncoder()
y=y_labelencoder.fit_transform(y)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [14]:
y=y.reshape(-1,1)
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y)
Y.shape

(103, 3)

In [19]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images,Y=shuffle(images,Y,random_state=1)

train_x,test_x,train_y,test_y =train_test_split(images,Y,test_size=0.1,random_state=415)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(92, 60, 60, 3)
(92, 3)
(11, 60, 60, 3)
(11, 3)


In [29]:
import tensorflow as tf

num_classes=3
num_channels = 3
img_size = 60
learning_rate=0.001

In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(img_size,img_size,num_channels)),
    tf.keras.layers.Conv2D(32,(5,5),activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64,(5,5),activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(128,(5,5),activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048,activation='relu'),
    tf.keras.layers.Dense(num_classes,activation='softmax')
])


In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 60, 60, 32)          │           2,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 30, 30, 64)          │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 15, 15, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 15, 15, 128)         │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2048)                │      12,847,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           6,147 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,111,875 (50.02 MB)

 Trainable params: 13,111,875 (50.02 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.fit(train_x, train_y, epochs=10, batch_size=16, validation_split=0.2)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 209ms/step - accuracy: 0.3271 - loss: 2.0206 - val_accuracy: 0.4211 - val_loss: 1.0938
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step - accuracy: 0.3889 - loss: 1.0996 - val_accuracy: 0.4211 - val_loss: 1.0903
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 0.3653 - loss: 1.0927 - val_accuracy: 0.4211 - val_loss: 1.0904
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - accuracy: 0.4293 - loss: 1.0855 - val_accuracy: 0.4211 - val_loss: 1.0711
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 0.4113 - loss: 1.0613 - val_accuracy: 0.5263 - val_loss: 1.0530
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 0.6380 - loss: 1.0103 - val_accuracy: 0.3158 - val_loss: 1.1637
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.4421 - loss: 1.0368 - val_accuracy: 0.5789 - val_loss: 0.9980
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.5828 - loss: 0.9336 - val_accuracy: 0.4211 - val_loss:

In [36]:
test_loss, test_accuracy = model.evaluate(test_x, test_y)
print("Test Accuracy:", test_accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.2727 - loss: 1.1779
Test Accuracy: 0.27272728085517883


In [37]:
predictions = model.predict(test_x)
predicted_classes = tf.argmax(predictions, axis=1).numpy()

true_classes = tf.argmax(test_y, axis=1).numpy()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
